In [27]:
import csv
import math
import os
from scipy.integrate import quad



def calcAvgSensorValue(sensorValues):
    avg = sum(sensorValues) / len(sensorValues)
    avg = round(avg, 9)
    return avg


def calcStdDev(sensorValues, meanVal):
    sd = []
    for num in sensorValues:
        result = num - meanVal
        result = result * result
        sd.append(result)
    return calcAvgSensorValue(sd)


def normalize(sensorValues):
    normalized_sensor = []
    for val in sensorValues:
        val = float(val)
        normalized = 2 * ((val - min(sensorValues)) / (max(sensorValues) - min(sensorValues))) - 1
        normalized_sensor.append(normalized)
    return normalized_sensor


def integral(i):
    return getGaussianVal(i, 0, 0.25)


def getGaussianVal(i, avg, sd):
    i = float(i - avg) / sd
    gauss = math.exp(-i * i / 2.0) / math.sqrt(2.0 * math.pi) / sd
    return gauss


def determineBands():
    numBands = r * 2
    bandList = []
    bandStart = -1
    for i in range(1, numBands):
        integral1, e = quad(integral, (i - r - 1) / r, (i - r) / r)
        integral2, e = quad(integral, -1, 1)
        length_i = 2 * (integral1 / integral2)
        band = bandStart + length_i
        bandList.append(band)
        bandStart = band
    bandList.append(1.0)
    return bandList


def quantize(values, bandList):
    quantized = ""
    for i in range(len(values)):
        bound = -1
        for band in bandList:
            if band >= values[i] > bound:
                quantized += str(bandList.index(band) + 1)
                break
            else:
                bound = band
    return quantized


def getWords():
    wordList = []
    i = 0
    while (i + w - 1) < len(quantizedSensor):
        word = quantizedSensor[i:i + w]
        wordList.append(word)
        i += s
    return wordList


def addToUniqueDict(word_tuple):
    inList = False
    for word in unique_dict:
        if word == word_tuple:
            inList = True
            break
    if not inList:
        unique_dict.append(word_tuple)


def calcAvgQuanAmp():
    avgQuanAmpList = []
    normWord = []
    i = 0
    while (i + w - 1) < len(normSensorVals):
        word = normSensorVals[i:i + w]
        normWord.append(word)
        i += s

    for word in normWord:
        avgAmp = sum(word) / len(word)
        avgQuanAmpList.append(avgAmp)

    return avgQuanAmpList


def getWordsFromFile(file):
    with open(file, "r") as f:
        words = []
        for line in f:
            line = line.strip()
            row = line.split(" - [")  # list of all words as they occur in the file
            if len(row) > 1:
                words.append(row[1])
        f.close()
    return formatWordsFromFile(words)


def formatWordsFromFile(tempList):
    allWords = []
    for sensorWords in tempList:
        sensorWords = sensorWords.replace("]", "")
        sensorWords = sensorWords.replace("'", "")
        wrd = sensorWords.split(", ")
        allWords.append(wrd)
    return allWords


def getUniqueWordsInGesture(allWordsInGesture):
    uniqueWords = []
    for i in range(len(allWordsInGesture)):  # i is each sensor in the gesture
        for word in allWordsInGesture[i]:
            uniqueWord = (direct, i+1, word)
            if uniqueWord not in uniqueWords:
                uniqueWords.append(uniqueWord)
    return uniqueWords


def calcTfValue(wordTuple):
    totalWords = 0
    sensorIndex = wordTuple[1]-1
    num_occurs = 0
    component = wordTuple[0]

    for file in os.listdir(directory + component):
        if file.endswith(".wrd"):
            file = directory + component + "/" + file
            gestFileWords = getWordsFromFile(file)
            num_occurs += gestFileWords[sensorIndex].count(wordTuple[2])
            totalWords += len(gestFileWords[sensorIndex])
    value = num_occurs / totalWords
    return value


def calcIdfValue(wordTuple):
    numObjs = 60
    numObjsWithWord = 1
    sensorIndex = wordTuple[1]-1
    component = wordTuple[0]

    for file in os.listdir(directory + component):
        if file != gestureFile and file.endswith(".wrd"):
            file = directory + component + "/" + file
            gestFileWords = getWordsFromFile(file)
            if word_tuple[2] in gestFileWords[sensorIndex]:
                numObjsWithWord += 1
                break

    value = math.log(numObjs / numObjsWithWord)
    return value


if __name__ == '__main__':
    # GLOBAL VARIABLES:
    unique_dict = []  # stores list of all unique words found
    gesture_dict = []  # stores list of all words found across all files

    # TASK 0
    # TASK 0A
    directory = input("Enter the data directory path: ")
    w = input("Enter the window length: ")
    s = input("Enter the shift length: ")
    r = input("Enter the resolution: ")

    w = int(w)
    s = int(s)
    r = int(r)

    # for each data file create a .wrd file containing the following:
    for direct in os.listdir(directory):
        # for each csv file in X,Y,W,Z:
        for filename in os.listdir(directory + direct):
            if filename.endswith(".csv"):
                bands = determineBands()

                # generate .wrd file
                with open(str(directory) + str(direct) + "/" + str(filename) + ".wrd", "w") as wrdFile:

                    sensor_id = 1
                    csvFile = open(str(directory) + str(direct) + "/" + filename, "r")
                    reader = csv.reader(csvFile, delimiter=',')
                    # for each sensor sj in file
                    for sensor in reader:
                        # output component ID, c in output file
                        wrdFile.write(str(direct) + ", ")

                        # write sensorID to wrd file
                        wrdFile.write(str(sensor_id) + ", ")

                        # compute and output average amplitude, avgij of the values
                        sensorVals = list(sensor)
                        sensorVals = [float(i) for i in sensorVals]
                        sensorAvg = calcAvgSensorValue(sensorVals)
                        wrdFile.write(str(sensorAvg) + ", ")

                        # compute and output standard deviations stdij of the values
                        stdDev = calcStdDev(sensorVals, sensorAvg)
                        wrdFile.write(str(stdDev) + ", ")

                        # normalize entries between -1 and 1
                        normSensorVals = normalize(sensorVals)

                        # quantizes entries into 2r levels as in phase 1
                        quantizedSensor = quantize(normSensorVals, bands)

                        # moves a w-length window on time series (by shifting it s units at a time), and at position h
                        sensorWords = getWords()

                        # computes and outputs in file average quantized amplitude avgQijh for window h of sensor sj
                        avgQuanAmp = calcAvgQuanAmp()
                        wrdFile.write(str(avgQuanAmp) + ", " + " - ")

                        # outputs symbolic quantized window descriptor winQijh for the window h of sensor sj
                        wrdFile.write(str(sensorWords) + "\n")

                        # add dictionary of each window to gestureDict list
                        for window in sensorWords:
                            wordDict = (direct, sensor_id, window)
                            gesture_dict.append(wordDict)
                            addToUniqueDict(wordDict)

                        sensor_id += 1
        # The dictionary of the words consists of <componentName, sensorID, winQ>

    # TASK 0B
    for direct in os.listdir(directory):
        # for each gesture file in W,X,Y,Z:
        # create vector .txt files with tf and tf-idf values
        for filename in os.listdir(directory + direct):
            if filename.endswith(".wrd"):
                tfFile = open(directory + direct + "/tf_vectors_" + filename[:-8] + ".txt", "w")
                tfidfFile = open(directory + direct + "/tfidf_vectors_" + filename[:-8] + ".txt", "w")
                gestureFile = directory + direct + "/" + filename

                allWordsInGesture = getWordsFromFile(gestureFile)
                uniqueWordsInGesture = getUniqueWordsInGesture(allWordsInGesture)

                for word_tuple in uniqueWordsInGesture:
                    tfValue = calcTfValue(word_tuple)
                    idfValue = calcIdfValue(word_tuple)
                    tf_idf_value = tfValue * idfValue
                    tfFile.write(str(word_tuple) + " - " + str(tfValue) + "\n")
                    tfidfFile.write(str(word_tuple) + " - " + str(tf_idf_value) + "\n")
    # End of TASK1



Enter the data directory path: Data/
Enter the window length: 2
Enter the shift length: 2
Enter the resolution: 2


In [4]:
#Task 1
import glob
import pandas as pd
import numpy as np
import pickle
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA

#unnecessary lines if you are using anaconda or another 
print("Please enter the following inputs as the same values you used for task 0: ")
directory = input("Enter the data directory path: ")
w = input("Enter the window length: ")
s = input("Enter the shift length: ")
r = input("Enter the resolution: ")

w = int(w)
s = int(s)
r = int(r)
#end of non-anaconda lines

#returns the top-k topics 
def PCAsetup(wordMat, k):
    #calculate PCA
    pca = PCA(k)
    pc = pca.fit_transform(wordMat)
    topK = pd.DataFrame(data = pc)
    
    return topK

def SVDsetup(wordMat, k):
    #calculate SVD
    svd =  TruncatedSVD(k)
    sv = svd.fit_transform(wordMat)
    topK = pd.DataFrame(data = sv)
    return topK

def NMFsetup(wordMat, k):
    #calculate NMF
    nmf = NMF(k)
    nm = nmf.fit_transform(wordMat)
    topK = pd.DataFrame(data = nm)
    return topK

def LDAsetup(wordMat, k):
    #calculate LDA
    lda = LDA(k)
    ld = lda.fit_transform(wordMat)
    topK = pd.DataFrame(data = ld)
    
    return topK


def makeMat(vectModel, axis):
        
    #read files
    if vectModel == "tf":
        Xmat = []
        for file in glob.glob(directory + axis + "/tf_vectors_*.txt"):
            #read tf file
            f = open(file, "r")
            tf_vectors = f.readlines()
        
            gestWords = []
            tfVals = []
        
            #split the line into the word and tf value
            for line in tf_vectors:
                noDash = line.split("-")
                tf_val = noDash[1]
                word = noDash[0].split(",")
                word[2] = word[2].replace(")","")
                word[2] = word[2].replace("'", "")
                word[2] = word[2].replace(" ", "")
                tf_val = tf_val.replace("\n", "")
                gestWords.append(word[2])
                tfVals.append(tf_val)
           
        
            index = 0
            startI = "1"
            for y in range(1, w):
                startI = startI + "1"
            startI = int(startI)
        
            #create dictionary with every word
            numWords = startI * (2*r) - startI
            wordMat = []
    
            for i in range(0, numWords + 1):
                wordMat.append(0)
            
            # put tf values into matrix where column = word
            for x in gestWords:
                xint = int(x)
            
                #if wordMat[xint - startI] == 0:
                wordMat[xint - startI] = wordMat[xint - startI] + float(tfVals[index])
                index = index + 1
                
            
            for iterate in range(0, len(wordMat)):
                wordMat[iterate] = wordMat[iterate] / 20

            
            Xmat.append(wordMat)
            f.close()
           
        return Xmat
    
    elif vectModel == "tfidf":
        Xmat = []
        for file in glob.glob(directory + axis + "/tfidf_vectors_*.txt"):
            #read tf file
            f = open(file, "r")
            tf_vectors = f.readlines()
        
            gestWords = []
            tfVals = []
        
            #split the line into the word and tf value
            for line in tf_vectors:
                noDash = line.split("-")
                tf_val = noDash[1]
                word = noDash[0].split(",")
                word[2] = word[2].replace(")","")
                word[2] = word[2].replace("'", "")
                word[2] = word[2].replace(" ", "")
                tf_val = tf_val.replace("\n", "")
                gestWords.append(word[2])
                tfVals.append(tf_val)
           
        
            index = 0
            startI = "1"
            for y in range(1, w):
                startI = startI + "1"
            startI = int(startI)
        
            #create dictionary with every word
            numWords = startI * (2*r) - startI
            wordMat = []
    
            for i in range(0, numWords + 1):
                wordMat.append(0)
            
            # put tf values into matrix where column = word
            for x in gestWords:
                xint = int(x)
            
                #if wordMat[xint - startI] == 0:
                wordMat[xint - startI] = wordMat[xint - startI] + float(tfVals[index])
                index = index + 1
                
            
            for iterate in range(0, len(wordMat)):
                wordMat[iterate] = wordMat[iterate] / 20

            
            Xmat.append(wordMat)
            f.close()
           
        return Xmat
    
def createdictofComponents(topk, k):
    #creates a matrix that contains tuples of the word and score
    outputMat = []
    outputMat = topk
    for i in range(0, k):
        word = "w" + str(i)
        for j in range(0, len(topk)):
            #print(topk[i][j])
            outputMat[i][j] = (topk[i][j], word)
            
    
    #sorts the words acording to their scores
    outputMat = np.transpose(outputMat)
    outputMat = outputMat.apply(lambda x: x.sort_values(ascending = False).values)
    outputMat = np.transpose(outputMat)
    finalMat = outputMat
    for i in range(0, k):
        for j in range(0, len(outputMat)):
            finalMat[i][j] = (outputMat[i][j][1], outputMat[i][j][0])
            
    file = open("./userOutput.txt", "w")
    file.write(str(finalMat))
    file.close()
    return finalMat
    
    
def task1(gestfiles, vectModel, useOp, k):
    
    if useOp == "PCA":
        #PCA for X axis
        wordMat = makeMat(vectModel, gestfiles)
        topk = PCAsetup(wordMat, k)
        print("\nPCA for " + gestfiles + " gesture files:\n")
        #print(topk)
        
        original_df = pd.DataFrame(topk)
        original_df.to_pickle("./PCA_" + gestfiles + "_" + vectModel + ".pkl")
        
        dictofComponents = createdictofComponents(topk, k)
        print(dictofComponents)
        
        
    elif useOp == "SVD":
        wordMat = makeMat(vectModel, gestfiles)
        topk = SVDsetup(wordMat, k)
        print("\nSVD for " + gestfiles + " gesture files:\n")
        #print(topk)
        
        original_df = pd.DataFrame(topk)
        original_df.to_pickle("./SVD_" + gestfiles + "_" + vectModel + ".pkl")
        
        dictofComponents = createdictofComponents(topk, k)
        print(dictofComponents)
        
        
    elif useOp == "NMF":
        wordMat = makeMat(vectModel, gestfiles)
        topk = NMFsetup(wordMat, k)
        print("\nNMF for " + gestfiles + " gesture files:\n")
        #print(topk)
        
        original_df = pd.DataFrame(topk)
        original_df.to_pickle("./NMF_" + gestfiles + "_" + vectModel + ".pkl")
        
        dictofComponents = createdictofComponents(topk, k)
        print(dictofComponents)

        
    elif useOp == "LDA":
        wordMat = makeMat(vectModel, gestfiles)
        topk = LDAsetup(wordMat, k)
        print("\nLDA for " + gestfiles + " gesture files:\n")
        #print(topk)
        
        original_df = pd.DataFrame(topk)
        original_df.to_pickle("./LDA_" + gestfiles + "_" + vectModel + ".pkl")
        
        dictofComponents = createdictofComponents(topk, k)
        print(dictofComponents)
        


gestfiles = input("Enter the folder that you want analyzed: ")
vectModel = input("Enter the vector model: ")
k = input("Enter k: ")
useOp = input("Enter the analysis you would like to use: ")
k = int(k)
task1(gestfiles, vectModel, useOp, k)

#p = pickle.load( open( "LDA_W_tf.pkl", "rb" ) )
#print("pickle")
#print(p)

#sample output: 
#Enter the folder that you want analyzed: X
#Enter the vector model: tf
#Enter k: 10
#Enter the analysis you would like to use: PCA



Please enter the following inputs as the same values you used for task 0: 
Enter the data directory path: Data/
Enter the window length: 3
Enter the shift length: 3
Enter the resolution: 3
Enter the folder that you want analyzed: Y
Enter the vector model: tfidf
Enter k: 5
Enter the analysis you would like to use: NMF

NMF for Y gesture files:



C:\Users\alexs\Anaconda3\lib\site-packages\ipykernel_launcher.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\alexs\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


                            0                          1  \
0    (w0, 0.4709964071350286)  (w1, 0.07001925978902977)   
1      (w0, 0.44519306966173)  (w2, 0.14411934697197035)   
2    (w0, 0.4565627673746854)  (w2, 0.12366068197651954)   
3    (w0, 0.4584927387442502)   (w2, 0.0752980964434521)   
4     (w0, 0.460972055525774)  (w2, 0.11497175480126079)   
5    (w0, 0.4497994488981324)  (w1, 0.14572989887619262)   
6    (w0, 0.4257617789060691)  (w2, 0.08854511092475649)   
7   (w0, 0.48231680136078176)  (w2, 0.07674545406671607)   
8    (w0, 0.4307465348071844)  (w2, 0.07687991635742807)   
9    (w0, 0.4307022416835115)  (w2, 0.17105516474491742)   
10  (w0, 0.48841472557392934)  (w2, 0.06498657199788545)   
11   (w0, 0.4277023535773086)  (w2, 0.17672257129333319)   
12  (w0, 0.43494113794588257)  (w2, 0.11879694729695554)   
13   (w0, 0.4322299795783147)   (w2, 0.1200200765022367)   
14   (w0, 0.4611232785897005)  (w1, 0.10606778544064502)   
15   (w0, 0.4527829035455592)   (w1, 0.1